In [2]:
import os

os.chdir('../..')

In [3]:
import glob
import os
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.ndimage.measurements import center_of_mass

import analysis.images as im

In [4]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [5]:
load_dir = os.path.join('data', 'kinect', 'labelled_trials')

best_pos_dir = os.path.join('data', 'kinect', 'best_pos')

In [6]:
labelled_trial_names = os.listdir(load_dir)

In [7]:
best_pos_dir = os.path.join('data', 'kinect', 'best_pos')

body_part_grayscale = {'HEAD': 105,
                     'R_FOOT': 76, 
                     'L_FOOT': 135}

# Camera calibration parameters
x_res, y_res = 565, 430
f_xz, f_yz = 1.11146664619446, 0.833599984645844


pattern = re.compile(r'(\d+)\.png')

for trial_name in labelled_trial_names[:1]:
    
    # Best positions selected by our method
    df_best_pos = pd.read_pickle(os.path.join(best_pos_dir, trial_name + '.pkl'))
    
    label_dir = os.path.join(load_dir, trial_name, 'label')
    depth_dir = os.path.join(load_dir, trial_name, 'depth16bit')

    df_estimated = pd.read_pickle(os.path.join(best_pos_dir, trial_name + '.pkl'))
    
    label_paths = sorted(glob.glob(os.path.join(label_dir, '*.png')))
    depth_paths = sorted(glob.glob(os.path.join(depth_dir, '*.png')))
    
    depth_filenames = [os.path.basename(x) for x in depth_paths]
    frames = [int(re.search(pattern, x).group(1)) for x in depth_filenames]
    
    for i, frame in enumerate(frames):

        
        label_path, depth_path = label_paths[i], depth_paths[i]
                        
        label_image = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        depth_image = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)
        
        binary_image = label_image == body_part_grayscale['HEAD']
        
        
        if np.any(binary_image):
        
            y, x = np.round(center_of_mass(binary_image)).astype(int)
            z = depth_image[y, x] / 10  # Convert mm to cm
            
            point_real = im.image_to_real([x, y, z], x_res, y_res, f_xz, f_yz)
            
                
        if frame == 411:
            break

In [8]:
frames_best_pos = df_best_pos.index.values
np.intersect1d(frames_best_pos, frames)

array([273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285,
       409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421,
       422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 555,
       556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568,
       569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581,
       582, 583, 584, 585, 586, 587, 588])

In [9]:
plt.figure()

# plt.imshow(label_image)
plt.imshow(depth_image)

plt.scatter(x, y, c='w', edgecolor='k')

plt.title('Frame {}'.format(frame))

Text(0.5, 1.0, 'Frame 411')

In [29]:
df_best_pos.loc[frame].HEAD

array([ 91.98013404,  59.17312589, 247.019     ])

In [30]:
point_real

array([149.13816834,  48.44146441, 242.6       ])

In [12]:
trial_name

'2014-12-03_P005_Post_002'

In [20]:
label_filenames = [os.path.basename(x) for x in label_paths]

depth_frames = [int(re.search(pattern, x).group(1)) for x in depth_filenames]
label_frames = [int(re.search(pattern, x).group(1)) for x in label_filenames]


In [21]:
label_frames == depth_frames

True

['data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00273.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00274.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00275.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00276.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00277.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00278.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00279.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00280.png',
 'data/kinect/labelled_trials/2014-12-03_P005_Post_002/depth16bit/14-12-03_trial-002_depth16bit00281.png',
 'data/kinect/labelled_trials/2014-12